In [1]:
import pandas as pd
import numpy as np
import os
import random
from glob import glob

In [2]:
has_path=os.path.exists

In [3]:
data_name = 'synthetic'
data_root_dir = '/home/hanshan/Data/DataCV'
info_path = f'{data_root_dir}/DataSets/{data_name}/info.csv'

In [4]:
has_path(info_path), info_path

(False, '/home/hanshan/Data/DataCV/DataSets/synthetic/info.csv')

### 定义 img， label，fixation 的路径

In [5]:
phase_dict = ['train', 'test'] 
# test只会生成image路径

In [6]:
phase = 'train'

In [7]:
img_path = f'{data_root_dir}/DataSets/{data_name}/{phase}/images'
label_path = f'{data_root_dir}/DataSets/{data_name}/{phase}/saliency'
fixation_path = f'{data_root_dir}/DataSets/{data_name}/{phase}/fixation'

In [10]:
img_path = f'{data_root_dir}/DataSets/{data_name}/images'
label_path = f'{data_root_dir}/DataSets/{data_name}/saliency'
fixation_path = f'{data_root_dir}/DataSets/{data_name}/fixation'

In [11]:
img_path, label_path, fixation_path

('/home/hanshan/Data/DataCV/DataSets/synthetic/images',
 '/home/hanshan/Data/DataCV/DataSets/synthetic/saliency',
 '/home/hanshan/Data/DataCV/DataSets/synthetic/fixation')

In [12]:
has_path(img_path), has_path(label_path), has_path(fixation_path)

(True, True, True)

# 1 生成 csv文件

### 获得所有图片的名字

In [13]:
img_file = '/*/*.*'
img_path + img_file

'/home/hanshan/Data/DataCV/DataSets/synthetic/images/*/*.*'

In [15]:
img_file = '/*.*'
img_path + img_file

'/home/hanshan/Data/DataCV/DataSets/synthetic/images/*.*'

In [16]:
len(glob(img_path + img_file)), len(glob(label_path + img_file)), len(glob(fixation_path+img_file))

(80, 80, 80)

## 拼接 img，label，fixation 的路劲

In [17]:
just_img = False
if phase == 'test':
    just_img = True

In [ ]:
# os package
# if just_img:
#     imgs =[ item for item in os.listdir(img_path) if not item.startswith('.')]
# else:
#     imgs =[ item for item in os.listdir(label_path) if not item.startswith('.')]
# len(imgs)

In [18]:
imgs = glob(img_path+img_file)
len(imgs)

80

In [22]:
items = []
for img in imgs:
    # print(img)
    # img_name = '_'.join(img.split('_')[:-1]) 
    # img_name= '/'.join(img.split('/')[-2:]).split('.')[0]
    img_name =  '/'.join(img.split('/')[-1:]).split('.')[0]
    # print(img_name)
    cur_img_path = img
    cur_label_path = os.path.join(label_path, img_name+'.png')
    cur_fixation_path = os.path.join(fixation_path, img_name+'.png')
    
    ## 检查拼接的路径
    if not has_path(cur_img_path):
        print('erro img', cur_img_path)
        continue
    
    if not just_img:
        if not has_path(cur_label_path):
            print('erro label', cur_label_path)
            continue
        if not has_path(cur_fixation_path):
            print('erro fix', cur_fixation_path)
            continue

    change = lambda x: '/'.join(x.split('/')[len(data_root_dir.split('/')):])
    
    cur_img_path = change(cur_img_path)
    
    if not just_img:
        cur_label_path = change(cur_label_path)
        cur_fixation_path = change(cur_fixation_path) 
    
        items.append(
            [cur_img_path, cur_label_path, cur_fixation_path]
        )
    else:
        items.append(
            [cur_img_path]
        )
print('如果只输出这一句话， info dataframe 是正确的')

如果只输出这一句话， info dataframe 是正确的


# 2 检查生成的路径

In [23]:
items[0]

['DataSets/synthetic/images/1.jpg',
 'DataSets/synthetic/saliency/1.png',
 'DataSets/synthetic/fixation/1.png']

In [24]:
for i in range(len(items)):
    for j in range(len(items[i])):
        if not has_path(os.path.join(
            data_root_dir, items[i][j]
        )):
            print(items[i][j])
print('如果只输出这一句话， info dataframe 是正确的')

如果只输出这一句话， info dataframe 是正确的


In [25]:
if not just_img:
    df = pd.DataFrame(items, columns=['img', 'saliency', 'fixation'])
else:
    df = pd.DataFrame(items, columns=['img'])

In [26]:
df.head()

,img,saliency,fixation
0,DataSets/synthetic/images/1.jpg,DataSets/synthetic/saliency/1.png,DataSets/synthetic/fixation/1.png
1,DataSets/synthetic/images/2.jpg,DataSets/synthetic/saliency/2.png,DataSets/synthetic/fixation/2.png
2,DataSets/synthetic/images/3.jpg,DataSets/synthetic/saliency/3.png,DataSets/synthetic/fixation/3.png
3,DataSets/synthetic/images/4.jpg,DataSets/synthetic/saliency/4.png,DataSets/synthetic/fixation/4.png
4,DataSets/synthetic/images/5.jpg,DataSets/synthetic/saliency/5.png,DataSets/synthetic/fixation/5.png


In [27]:
df.to_csv(info_path.replace('info', phase), index=False)

# 3 生成 train valid test csv

In [ ]:
nums = len(df)

In [ ]:
shuffle_index = random.sample(range(nums), nums)

In [ ]:
train_num = int(nums*0.6)
valid_num = int(nums*0.2)

In [ ]:
df_train = df.iloc[:train_num, :]
df_valid = df.iloc[train_num:train_num+valid_num, :]
df_test = df.iloc[train_num+valid_num:, :]

In [ ]:
len(df_train), len(df_valid), len(df_test)

In [ ]:
df_train.head()

In [ ]:
data_csv_dir = os.path.join(data_root_dir,
                           'CSVs', data_name)
if not os.path.exists(data_csv_dir):
    os.makedirs(data_csv_dir)

In [ ]:
save_csv = lambda df, phase: df.to_csv(
            data_root_dir + 'CSVs/'+data_name+'/'+phase+'.csv', index=False
)

In [ ]:
if not has_path('CSVs/'+data_name):
    os.makedirs('CSVs/'+data_name)
save_csv(df_train, 'train')
save_csv(df_valid, 'valid')
save_csv(df_test, 'test')